In [ ]:
from os.path import expanduser
import pickle
import time

In [ ]:
from clang.cindex import Index, Config, TranslationUnit

In [ ]:
from pygccxml import declarations
from pygccxml import utils
from pygccxml import parser

In [ ]:
castxml_git = expanduser("~/devel/CastXML")
castxml_file = f"{castxml_git}/src/Output.cxx"

llvm_dir = "/usr/lib/llvm-9"

In [ ]:
if not Config.loaded:
    Config.set_library_file(f"{llvm_dir}/lib/libclang.so")

index = Index.create()

t_start = time.time()
tu = TranslationUnit.from_source(
    filename=castxml_file,
    index=index,
    args=[
        f"-I{llvm_dir}/include",
        f"-I{castxml_git}/src",
    ],
)
dt = time.time() - t_start
print(dt)  # About 3.6s

In [ ]:
# See: https://github.com/llvm/llvm-project/tree/master/clang/bindings/python/examples/cindex
def visit(f, node):
    f(node)
    for c in node.get_children():
        visit(f, c)

def my_func(node):
    if node.spelling == "hasAttr":  
#     if "hasAttr" in node.spelling:
        print(node.spelling, node.kind, node.location)
        tokens = tu.get_tokens(extent=node.extent)
        print([t.spelling for t in tokens])

visit(my_func, tu.cursor)

In [ ]:
# Find out the c++ parser. This should resolve to the castxml
# version installed in Docker.
generator_path, generator_name = utils.find_xml_generator()

# Use source version built against system llvm / libclang.
generator_path = f"{castxml_git}/build_sys/bin/castxml"

declarations.scopedef_t.ALLOW_EMPTY_MDECL_WRAPPER = True

# Configure the xml generator
config = parser.xml_generator_configuration_t(
    xml_generator_path=generator_path,
    xml_generator=generator_name,
    include_paths=[
        f"{llvm_dir}/include",
        f"{castxml_git}/src",
    ],
    keep_xml = True,
    # TODO(eric.cousineau): This doesn't work?
    flags=["-Wno-unused-value"],
)
# This is the wrong cache setup... Need to have it save to *.xml file? :(
cache = parser.declarations_cache.file_cache_t("/tmp/pygccxml_cache.pkl")

In [ ]:
t_start = time.time()
decls = parser.parse(
    files=[castxml_file],
    config=config,
    compilation_mode=parser.COMPILATION_MODE.ALL_AT_ONCE,
    cache=cache,
)
dt = time.time() - t_start
print(dt)  # 69.7s

In [ ]:
# # Can't dump :(
# with open("/tmp/pygccxml_custom_cache.pkl", "w") as f:
#     pickle.dump(decls, f)

In [ ]:
cache.flush()

In [ ]:
(global_ns,) = decls

t_start = time.time()
global_ns.init_optimizer()
dt = time.time() - t_start
print(dt)  # 30s, but then other queries are hella fast

In [ ]:
clang = global_ns.namespace("clang")

In [ ]:
len(clang.classes())

In [ ]:
# N.B. clang::Decl does not have 
cls = clang.class_("Decl")

In [ ]:
declarations.print_declarations([cls])

In [ ]:
for i, cls in enumerate(clang.classes()):
    if i > 10:
        break
    print(cls.name)

In [ ]:
t_start = time.time()
cls = global_ns.class_("::clang::ImportDecl")
dt = time.time() - t_start
print(dt)
# Without `init_optimizer`, 37s first time, 0.75s after.
# With `init_optimizer`, 0.3ms!

In [ ]:
declarations.print_declarations(cls)

In [ ]:
# Cache looks super small? 14 bytes?
cache.flush()